In [1]:
import psycopg2
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
#import constants
from datetime import date, datetime, timedelta
load_dotenv()
pd.options.mode.chained_assignment = None

from dateutil.relativedelta import relativedelta


In [4]:
date_col = ['Ngày ĐĐH','Ngày CT','Ngày dự tính giao hàng ban đầu','Ngày dự định giao hàng']

In [6]:
for col in date_col:
    df[col] = pd.to_datetime(df[col], dayfirst=True)

In [10]:
df1 = df[df['Ngày dự định giao hàng']>='2025-04-20']

In [11]:
df_grouped = df1.groupby('KH Tên gọi tắt').agg({'SL đơn đặt':'sum'}).reset_index()

In [ ]:
df_grouped.sort_values(by='SL đơn đặt', ascending=False)

In [ ]:
import pytz
a = pytz.all_timezones
for i in a:
    if 'Asia' in i:
        print(i)

In [2]:
from utils.db import Database

In [3]:
Database.initialize(
    database=os.getenv('WAREHOUSE_NAME'),
    user=os.getenv('WAREHOUSE_USER'),
    password=os.getenv('WAREHOUSE_PASSWORD'),
    host=os.getenv('WAREHOUSE_HOST'),
    port=os.getenv('WAREHOUSE_PORT'),
    minconn=1,
    maxconn=20
)

In [4]:
conn = Database.get_connection()

In [5]:
cur = conn.cursor()

In [11]:
factory_name='大森 TIM BER'

In [12]:
selected_year = 2025

In [13]:
from utils.query.wh.extract import get_mtd_by_month, get_factory_code

In [14]:
factory_code = get_factory_code(factory_name)
# Get sales
df_sales = get_mtd_by_month(selected_year, 1, 31, factory_code, "fact_sales", "sales_quantity", "sales_date")
df_sales.columns = ['month','sales_quantity','sales_quantity_timber']
df_sales.drop(columns = ['sales_quantity'], inplace=True)


In [15]:
df_sales

,month,sales_quantity_timber
0,1,25590.0


In [16]:

# Get planned deliveries
df_plan  = get_mtd_by_month(selected_year, 1, 31, factory_code, "fact_order", "order_quantity", "estimated_delivery_date")
df_plan.columns = ['month','plan_quantity','plan_quantity_timber']
df_plan.drop(columns = ['plan_quantity'], inplace=True)


In [17]:
df_plan

,month,plan_quantity_timber
0,1,40697.0
1,2,33976.0
2,9,0.0


In [18]:
conn = Database.get_connection()
cur = conn.cursor()

In [29]:
cur.execute("""DELETE from fact_order where order_code = '2201-240829022-0006'""")

In [30]:
conn.commit()

In [27]:
data = cur.fetchall()
column_names = [description[0] for description in cur.description]
df = pd.DataFrame(data=data, columns=column_names)

In [28]:
df

,order_date,order_code,ct_date,factory_code,factory_order_code,tax_type,department,salesman,deposit_rate,payment_registration_code,...,package_order_quantity,delivered_package_order_quantity,unit,package_unit,estimated_delivery_date,original_estimated_delivery_date,pre_ct,finish_code,import_timestamp,import_wh_timestamp
0,2024-08-29,2201-240829022-0006,2024-08-29,31095,DHM-24/08-091,Thuế cộng riêng,業務部 NV,陳國勇,0,2004.0,...,1.0,1.0,kg,T,2025-09-04,2024-08-30,2101-220401022-0099,T? ??ng k?t thuc,2024-12-03 11:30:23.803593,2024-12-03 13:07:25.288145


In [25]:
df.to_excel('temp.xlsx',index=False)

In [ ]:

# Merge for plotting
df = df_sales.merge(df_plan, on='month', how='outer')
df.columns = ['month','sales_quantity','plan_quantity']

df.fillna(0,inplace=True)

df['percentage'] = df.apply(

    lambda row: 100 if row['plan_quantity'] == 0 else 
                0 if row['sales_quantity'] == 0 else 
                (row['sales_quantity'] / row['plan_quantity'] * 100),
    axis=1
)

In [1]:
import pandas as pd
import plotly.express as px

In [2]:
d1 = dict(Start= '2021-10-10 02:00:00', Finish = '2021-10-10 09:00:00', Task = 'Sleep')
d2 = dict(Start= '2021-10-10 09:00:00', Finish = '2021-10-10 09:30:00', Task = 'EAT')
d3 = dict(Start= '2021-10-10 09:30:00', Finish = '2021-10-10 12:00:00', Task = 'Study')
d4 = dict(Start= '2021-10-10 12:00:00', Finish = '2021-10-10 16:00:00', Task = 'Work')
d5 = dict(Start= '2021-10-10 16:00:00', Finish = '2021-10-10 16:50:00', Task = 'EAT')
d6 = dict(Start= '2021-10-10 17:00:00', Finish = '2021-10-10 20:00:00', Task = 'Study')
d8 = dict(Start= '2021-10-10 20:00:00', Finish = '2021-10-10 20:40:00', Task = 'EAT')
d7 = dict(Start= '2021-10-10 21:00:00', Finish = '2021-10-11 05:00:00', Task = 'Sleep')

df = pd.DataFrame([d1,d2,d3,d4,d5,d6,d7,d8])

In [7]:
gantt = px.timeline(df, x_start='Start', x_end = 'Finish', color = 'Task', height=150, y=['']*df.shape[0]) 
gantt.show()

In [1]:
import pandas as pd
import plotly.express as px

In [7]:
from utils.query.wh.extract import get_sales_ratio
df = get_sales_ratio(2025)

In [18]:
df

,sales_quantity,factory_code,factory_name,year,month,product_type
0,32.0,30301,凱勝 KAISER 1,2025,3,烤調色PM HAP
1,210.0,30622,泓一 HONG YI,2025,3,原料溶劑 NL DUNG MOI
2,1200.0,30992,大顺 ĐẠI THUẬN,2025,3,面漆 BONG
3,70.0,30070,大皇美 ĐẠI HOÀNG MỸ,2025,4,底漆 LOT
4,16.0,30227,維成 DUY THÀNH,2025,1,烤調色PM HAP
...,...,...,...,...,...,...
2471,32.0,30724,采岩 TSAI YARN,2025,2,面漆 BONG
2472,90.0,30011,上順 SANG SHUN,2025,7,半成品BAN THANHPHAM
2473,7440.0,30301,凱勝 KAISER 1,2025,2,補土 BOT TRET
2474,14700.0,30449,信亞 HSIN YA,2025,3,成品溶劑DUNG MOI TP


In [9]:
all_factories = df[['factory_code', 'factory_name']].drop_duplicates()
all_months = sorted(df['month'].unique())

In [11]:
df_thinner = df[df['product_type'].isin(['原料溶劑 NL DUNG MOI','成品溶劑DUNG MOI TP'])]
df_paint = df[df['product_type'].isin(['色母SON CAI','木調色PM GO',
                                    '烤調色PM HAP','底漆 LOT','面漆 BONG'])]

In [20]:
thinner_types = ['原料溶劑 NL DUNG MOI', '成品溶劑DUNG MOI TP']
paint_types = ['色母SON CAI', '木調色PM GO', '烤調色PM HAP', '底漆 LOT', '面漆 BONG']


In [21]:

# Get all combinations of factory and month (to ensure 0 filling)
factories_months = df[['factory_code', 'factory_name', 'month']].drop_duplicates()


In [ ]:

# ---- DF_THINNER ----
df_thinner = (
    df[df['product_type'].isin(thinner_types)]
    .groupby(['factory_code', 'factory_name', 'month'], as_index=False)['sales_quantity']
    .sum()
)

In [ ]:

df_thinner = factories_months.merge(df_thinner, 
                                    on=['factory_code', 'factory_name', 'month'], 
                                    how='left').fillna({'sales_quantity': 0})

In [32]:

# ---- DF_PAINT ----
df_paint = (
    df[df['product_type'].isin(paint_types)]
    .groupby(['factory_code', 'factory_name', 'month'], as_index=False)['sales_quantity']
    .sum()
)

df_paint = factories_months.merge(df_paint, 
                                  on=['factory_code', 'factory_name', 'month'], 
                                  how='left').fillna({'sales_quantity': 0})

In [35]:
thinner_df = df_thinner.pivot(index=['factory_code','factory_name'],
                              columns='month',
                              values='sales_quantity')
    
paint_df = df_paint.pivot(index=['factory_code','factory_name'],
                            columns='month',
                            values='sales_quantity')
ratio_df = paint_df.div(thinner_df.replace(0, np.nan)).fillna(0)

In [36]:
thinner_df

,month,1,2,3,4,5,6,7
factory_code,factory_name,,,,,,,
30003.2,林源 LÂM NGUYÊN,0.0,NaN,NaN,NaN,NaN,NaN,0.0
30008.4,鴻霖 HỒNG LÂM,NaN,360.0,NaN,NaN,360.0,360.0,NaN
30011,上順 SANG SHUN,4148.0,9212.0,5596.0,2176.0,5792.0,5988.0,6136.0
30013,協楊 ASY,9180.0,5790.0,9000.0,8280.0,9900.0,18225.0,11355.0
30020,楊林 YANG LIN,269.0,90.0,180.0,270.0,269.5,270.0,150.0
...,...,...,...,...,...,...,...,...
31329,LEATHER HOUSE VIETNAM,0.0,NaN,0.0,0.0,0.0,0.0,0.0
31336,TENGDA 騰逹五金,NaN,NaN,NaN,NaN,NaN,30.0,NaN
31337,麦林 SEVILLE SIGNATURE,NaN,NaN,NaN,NaN,180.0,NaN,NaN
